# Policy Gradient Agent 

### Importing libraries

In [1]:
import numpy as np

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from tensorflow.python.framework import ops

Instructions for updating:
non-resource variables are not supported in the long term


### The Bandits

In [2]:
bandits = [0.2,0,-0.2,-5]
num_bandits = len(bandits)

In [10]:
# Method to get the reward
def pullBandit(bandit):
    # Get a random number
    result = np.random.randn(1)
    
    if result > bandit:
        return 1
    else:
        return -1

### The Agent

In [4]:
ops.reset_default_graph()

In [6]:
# Feed forward network
weights = tf.Variable(tf.ones([num_bandits]))
chosen_actions = tf.argmax(weights,0)

In [8]:
# Back propogation and training
reward_holder = tf.placeholder(shape=[1], dtype=tf.float32)
action_holder = tf.placeholder(shape=[1], dtype=tf.int32)
responsible_weight = tf.slice(weights, action_holder, [1])

loss = -(tf.log(responsible_weight) * reward_holder)
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.001)
update = optimizer.minimize(loss)

### Training the agent

In [15]:
total_episodes = 1000
total_reward = np.zeros(num_bandits)
e = 0.1

init = tf.initialize_all_variables()

with tf.Session() as sess:
    sess.run(init)
    
    i = 0
    
    while i < total_episodes:
        if np.random.rand(1) < e:
            action = np.random.randint(num_bandits)
        else:
            action = sess.run(chosen_actions)
                    
        # Get the reward from the bandits
        reward = pullBandit(bandits[action])
        
        # Update the network
        _, resp, ww = sess.run([update, responsible_weight, weights], feed_dict={reward_holder:[reward], action_holder:[action]})
        
        # Update our running tally of scores
        total_reward[action] += reward
        
        if i % 50 == 0:
            print("Running reward for the " + str(num_bandits) + " bandits " + str(total_reward))
            
        i += 1
        
        

Running reward for the 4 bandits [-1.  0.  0.  0.]
Running reward for the 4 bandits [-1.  1.  0.  1.]
Running reward for the 4 bandits [-1.  1.  0. 27.]
Running reward for the 4 bandits [ 1.  2. -1. 73.]
Running reward for the 4 bandits [ -1.   0.  -1. 119.]
Running reward for the 4 bandits [  0.  -1.  -1. 167.]
Running reward for the 4 bandits [  0.  -1.   0. 214.]
Running reward for the 4 bandits [  0.   1.   1. 257.]
Running reward for the 4 bandits [  0.  -1.   1. 303.]
Running reward for the 4 bandits [  0.  -2.   1. 348.]
Running reward for the 4 bandits [  1.  -2.   0. 396.]
Running reward for the 4 bandits [  2.  -2.  -1. 440.]
Running reward for the 4 bandits [  2.   0.  -2. 487.]
Running reward for the 4 bandits [  1.   3.  -1. 532.]
Running reward for the 4 bandits [  0.   3.  -2. 578.]
Running reward for the 4 bandits [  0.   3.   1. 621.]
Running reward for the 4 bandits [  0.   1.   2. 668.]
Running reward for the 4 bandits [  0.   1.   3. 715.]
Running reward for the 4 b

In [16]:
print("The agent thinks bandit " + str(np.argmax(ww)+1) + " is the most promising....")
if np.argmax(ww) == np.argmax(-np.array(bandits)):
    print("...and it was right!")
else:
    print("...and it was wrong!")

The agent thinks bandit 4 is the most promising....
...and it was right!
